In [1]:
Dataset = 'Sales Feb.xlsx'

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from mlxtend.preprocessing import TransactionEncoder


In [3]:
df_o = pd.read_excel(Dataset)
df_o.head()

,Branch,رقم الفاتوره,فريق المبيعات,Barcode,Product,Product Category,Quantity,Discount per Unit (abs),Discount (%),Unit Price,Cost,Subtotal,Created on,Order Ref/Cashier
0,فروع محافظة بريدة,Order 19489-002-0014,مبيعات نقاط بيع راحتي بريدة الكبير 2,220102020264,[0823006CH2206] لحاف شتوي نفرين 6 قطع CH2206 شد 3,محلي-22 / مفروشات محلي-2201 / مفروشات محلى -2...,1.0,85.0,0,425.0,210.0,340.0,2024-02-01 00:00:00,مدحت رضا عبدالرحمن عبدالحليم
1,فروع محافظة بريدة,Order 19489-002-0014,مبيعات نقاط بيع راحتي بريدة الكبير 2,170-0050,[025] مخدة موج البحر مضغوط,محلي-22 / مفروشات محلي-2201 / مخدات محلى -220...,2.0,8.0,0,31.0,30.0,46.0,2024-02-01 00:00:00,مدحت رضا عبدالرحمن عبدالحليم
2,راحتى حفر الباطن,Order 19464-004-0095,مبيعات نقاط بيع راحتي حفر الباطن,575730,[575730] تركيب مجاني,خدمات-44 / خدمات-4401 / خدمات-440101 / خدمات 4...,-1.0,0.0,0,0.0,0.0,0.0,2024-02-01 00:03:55,مصطفي شوقي حلمي علي
3,راحتى حفر الباطن,Order 19464-004-0095,مبيعات نقاط بيع راحتي حفر الباطن,DLV20,[DLV20] خدمة توصيل,All,-1.0,0.0,0,20.0,0.0,-20.0,2024-02-01 00:03:55,مصطفي شوقي حلمي علي
4,راحتى حفر الباطن,Order 19464-004-0095,مبيعات نقاط بيع راحتي حفر الباطن,DLV10,[DLV10] خدمة توصيل,All,-1.0,0.0,0,10.0,0.0,-10.0,2024-02-01 00:03:55,مصطفي شوقي حلمي علي


In [4]:
df = df_o.copy()
df.head()

,Branch,رقم الفاتوره,فريق المبيعات,Barcode,Product,Product Category,Quantity,Discount per Unit (abs),Discount (%),Unit Price,Cost,Subtotal,Created on,Order Ref/Cashier
0,فروع محافظة بريدة,Order 19489-002-0014,مبيعات نقاط بيع راحتي بريدة الكبير 2,220102020264,[0823006CH2206] لحاف شتوي نفرين 6 قطع CH2206 شد 3,محلي-22 / مفروشات محلي-2201 / مفروشات محلى -2...,1.0,85.0,0,425.0,210.0,340.0,2024-02-01 00:00:00,مدحت رضا عبدالرحمن عبدالحليم
1,فروع محافظة بريدة,Order 19489-002-0014,مبيعات نقاط بيع راحتي بريدة الكبير 2,170-0050,[025] مخدة موج البحر مضغوط,محلي-22 / مفروشات محلي-2201 / مخدات محلى -220...,2.0,8.0,0,31.0,30.0,46.0,2024-02-01 00:00:00,مدحت رضا عبدالرحمن عبدالحليم
2,راحتى حفر الباطن,Order 19464-004-0095,مبيعات نقاط بيع راحتي حفر الباطن,575730,[575730] تركيب مجاني,خدمات-44 / خدمات-4401 / خدمات-440101 / خدمات 4...,-1.0,0.0,0,0.0,0.0,0.0,2024-02-01 00:03:55,مصطفي شوقي حلمي علي
3,راحتى حفر الباطن,Order 19464-004-0095,مبيعات نقاط بيع راحتي حفر الباطن,DLV20,[DLV20] خدمة توصيل,All,-1.0,0.0,0,20.0,0.0,-20.0,2024-02-01 00:03:55,مصطفي شوقي حلمي علي
4,راحتى حفر الباطن,Order 19464-004-0095,مبيعات نقاط بيع راحتي حفر الباطن,DLV10,[DLV10] خدمة توصيل,All,-1.0,0.0,0,10.0,0.0,-10.0,2024-02-01 00:03:55,مصطفي شوقي حلمي علي


# Cleaning Part

In [5]:
df[['Product origin', 'Cat1', 'cat2', 'cat3']]= df['Product Category'].str.split("/",expand=True)
df.drop(columns= {'Product Category'}, inplace = True)

In [6]:
df['day'] = pd.to_datetime(df['Created on']).dt.day
d = list(df['day'].dropna().unique())

df['month'] = pd.to_datetime(df['Created on']).dt.month_name()
m = list(df['month'].dropna().unique())

df['year'] = pd.to_datetime(df['Created on']).dt.year
y = list(df['year'].dropna().unique())

In [7]:
df['discount amount'] = df['Quantity'] * df['Discount per Unit (abs)']
df['discount percentage'] = ((df['Discount (%)'] / 100) * df['Unit Price']) *df['Quantity']
df['Total discount'] = df['discount amount']  + df['discount percentage']
df['price before discount'] = df['Unit Price'] * df['Quantity']
df['Cost per unit'] = df['Cost'] * df['Quantity']
df['vat'] = df['Subtotal'] / 1.15
df['profit margin'] = df['vat'] - df['Cost'] 
df['Profit margin percentage'] = df['profit margin'] /df['vat']

In [8]:
df['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'راحتي الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'فرع بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'راحتي الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'راحتي  الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'راحتي حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : ' راحتي الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : ' راحتي بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'راحتي حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'بيع راحتي حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'راحتي البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'راحتي الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'راحتي الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

### Prepare different pages 



In [9]:
df_deposit = df[df['Product origin'] == 'منتجات مصروفات الشركة']
df_deposit.to_excel('Service and other things report.xlsx', sheet_name='منتجات مصروفات الشركة',  index = False)
df.loc[df_deposit.index,"Quantity"] = 1

In [10]:
df_services = df[df['Product origin'] == 'خدمات-44 ']

In [11]:
df_installment = df[df['Product origin'] == 'خدمات التقسيط']
df_services =pd.concat([df_services, df_installment])

In [12]:
df_all = df[df['Product origin'] == 'All']
df_addition_discount = df_all[df_all['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all.drop(df_addition_discount.index, inplace = True)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_5068\266319778.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop(df_addition_discount.index, inplace = True)


In [13]:
df_0 = df[df['Quantity'] == 0 ]
df.drop(df_0.index, inplace = True)

In [14]:
ind = list(df[df['Product origin'] == 'All'].index)


In [15]:
df2= df.copy()
ind = list(df[df['Product origin'] == 'All'].index)
df2.drop(df_deposit.index, inplace = True, errors = 'ignore')
df2.drop(df_services.index, inplace = True, errors = 'ignore')
df2.drop(df_addition_discount.index, inplace = True, errors = 'ignore')
df2.drop(ind, inplace = True, errors = 'ignore')



# Analytical Part

## Based on invoice 


In [16]:
x = pd.DataFrame(df2.groupby(['Created on','رقم الفاتوره'])['Product'].apply(lambda x: list(x)))
transactions = x['Product'].values.tolist()

In [19]:
list1 = []
for n in transactions:
    list1.append(len(n))

In [25]:
sum(list1)/len(list1)

1.559586854460094

In [29]:
df2.columns

Index(['Branch', 'رقم الفاتوره', 'فريق المبيعات', 'Barcode', 'Product',
       'Quantity', 'Discount per Unit (abs)', 'Discount (%)', 'Unit Price',
       'Cost', 'Subtotal', 'Created on', 'Order Ref/Cashier', 'Product origin',
       'Cat1', 'cat2', 'cat3', 'day', 'month', 'year', 'discount amount',
       'discount percentage', 'Total discount', 'price before discount',
       'Cost per unit', 'vat', 'profit margin', 'Profit margin percentage'],
      dtype='object')

In [37]:

df_sales = pd.pivot_table(df,index=['رقم الفاتوره'], values=(['Subtotal']), aggfunc = 'sum')
df_sales.mean()

Subtotal    524.306191
dtype: float64

In [78]:
len(transactions)

26625

In [52]:
temp = TransactionEncoder()
temp_df = temp.fit(transactions).transform(transactions)
new_df = pd.DataFrame(temp_df,columns=temp.columns_)
new_df

,تحفة هدايا THY1179T- G,حامل زرع كبير HZ28111,حامل زرع كبير HZ28143,حامل زرع وسط HZ28001,حامل زرع وسط HZ28111,حامل زرع وسط HZ28143,مناديل سفرة 40سم 201GT40-2,( 00217797-60217799) كرسي بلاستيك,(10192824-10340) وحدة ادراج 36*70سم,DT-58*100 فرو ارضى,...,مفرش لميس مخمل 6ق نفرين RS-LAMES-21,مفرش ليلي ميكروفيبر نفرين 8ق RS-07,مفرش هيفاء شتوي مخمل 6ق نفرين RS-Haifa-01,مكنسة كهربائية شحن,مناشف ساندى مصرى40*70,مناشف ماركوبولو تركي 50*90 سم,مناشف ماركوبولو تركي 70*140سم,منشفة بانتكس مربعات وفرع مشجر 90*180,وحدة درج مع عجلات ابيض / أسود ‎28x69 سم‏ 10251045 - 20341970,وحدة رفوف 1 درج سلم SL-17013
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26620,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26621,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26622,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26623,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [62]:
from mlxtend.frequent_patterns import apriori, association_rules
support_values = apriori(new_df, use_colnames=True)
support_values

,support,itemsets


In [85]:
from apyori import apriori
rules = apriori(transactions, min_support=0.00010,min_confidence = 0.05, min_length = 3)
results = list(rules)
results

[RelationRecord(items=frozenset({'YWH-75-15 شموع ', 'YWH-75-12 شموع '}), support=0.00022535211267605634, ordered_statistics=[OrderedStatistic(items_base=frozenset({'YWH-75-12 شموع '}), items_add=frozenset({'YWH-75-15 شموع '}), confidence=0.4615384615384615, lift=682.6923076923076), OrderedStatistic(items_base=frozenset({'YWH-75-15 شموع '}), items_add=frozenset({'YWH-75-12 شموع '}), confidence=0.33333333333333337, lift=682.6923076923077)]),
 RelationRecord(items=frozenset({'شمع مضيئ كبير YWH-75-20  ', 'YWH-75-12 شموع '}), support=0.00011267605633802817, ordered_statistics=[OrderedStatistic(items_base=frozenset({'YWH-75-12 شموع '}), items_add=frozenset({'شمع مضيئ كبير YWH-75-20  '}), confidence=0.23076923076923075, lift=1228.8461538461538), OrderedStatistic(items_base=frozenset({'شمع مضيئ كبير YWH-75-20  '}), items_add=frozenset({'YWH-75-12 شموع '}), confidence=0.6, lift=1228.8461538461538)]),
 RelationRecord(items=frozenset({'شمع مضيئ كبير YWH-75-20  ', 'YWH-75-15 شموع '}), support=0.00

In [86]:
def inspect(results):
    lhs         = [tuple(result[2][0][0]) for result in results]
    rhs         = [tuple(result[2][0][1]) for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
ordered_results = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [87]:
ordered_results.sort_values(by =['Confidence', 'Lift'], ascending = False).head(5)

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
106,"([05300429] كرسي ازرق 291 BLUE,)","([05300431] كرسي YELLOW291,)",0.000113,1.000,8875.000000
150,"([1240255] دولاب احذية SC GX23,)","([CY-673] كرسي - CY-673 - Red,)",0.000113,1.000,4437.500000
284,"([M05-S-76] مفرش مايكرو نفر ونص 5 قطع M05-S-76,)","(سجاد راحتي هناي - 100*200,)",0.000113,1.000,633.928571
511,"([D2303] بطانية راحتي - 200*230 - D2303, [6281...","([G2303] بطانية راحتي جاكار - 200*230 - G2303,)",0.000113,1.000,82.430341
210,"([BBE-08] سرير أطفال - BBE-08,)","([M0459] مرتبة هيبي مون 60*120*10سم M0459,)",0.000263,0.875,206.167035


In [88]:
rhs         = [tuple(result[2][0][1]) for result in results]
rhs

[('YWH-75-15 شموع ',),
 ('شمع مضيئ كبير YWH-75-20  ',),
 ('شمع مضيئ كبير YWH-75-20  ',),
 ('[3824] سرير راحتى غصن ابيض 120*200 موديل 3824',),
 ('[00030104] طاولة طعام خشب 8 كرسي G009 GREY',),
 ('[00030104] طاولة طعام خشب 8 كرسي G009 GREY',),
 ('[00030104] طاولة طعام خشب 8 كرسي G009 GREY',),
 ('[00030104] طاولة طعام خشب 8 كرسي G009 GREY',),
 ('[00030125] طاولة طعام رخام صناعي 6 كرسي T-001 GREY',),
 ('[1150310] دولاب بوفية رصاصي اسود  CM ZM955CN-16',),
 ('[1150293] بوفية رصاصي CM ZM2335C',),
 ('[SF-11086] ابجورات استاند ذهبي SF-11086',),
 ('[YK-2003] خزانة راحتي  بـدرجين, أبيض, \u200e40x55 سم\u200f -YK-2003',),
 ('[YK-3008] خزانة راحتي 3 درج أبيض 80*78سم -YK-3008',),
 ('[الشرابية] سرير خشب شرائح  120*190 ',),
 ('[TV4806P37] بلازما 1.80سم TV4806P37 BER37',),
 ('[TVZ0618CIG] بلازما 1.80سم TVZ0618CIG C.I.Grey',),
 ('[ZW003-008] مرايا - (62*52) -  ZW003-008',),
 ('سجاد راحتي هناي -  150*150',),
 ('[00070043] كنسول ستيل زجاج BS-1579 GOLD',),
 ('[00070098] كنسول خشب مع زجاج C255 GREY',),
 ('[0

In [83]:
ordered_results.to_excel('data13.xlsx')

In [44]:
count = [0 for i in range(0, ordered_results.shape[0])]
for i in ordered_results.index:
    for t in transactions:
            if ordered_results.iloc[i, 0] in t:
                if ordered_results.iloc[i, 1] in t:
                    count[i] +=1

                
x =  count[0: ordered_results.shape[0]]
ordered_results['times together algorithm'] = x

In [45]:
ordered_results['times in invoice together'] = round(ordered_results['Support'] * len(transactions))
ordered_results['times in invoice together'] = ordered_results['times in invoice together'].astype(int)

In [46]:
ordered_results.to_excel('Market.xlsx', sheet_name = 'data')

In [47]:
with pd.ExcelWriter('Market.xlsx', engine='openpyxl', mode='a') as writer:
    ordered_results.sort_values(by =['Confidence', 'Lift'], ascending = False).to_excel(writer, sheet_name= 'results', index = False)

## Based On Day of the week

In [ ]:
df['day'] = df['Created on'].dt.day_name() 
df['day']

In [ ]:
x = df.groupby(['Created on','رقم الفاتوره'])['day/product'].apply(lambda x: list(x))
x

In [ ]:
x = pd.DataFrame(x)

In [ ]:
newdata = df[['Product', 'day']].values
newdata = [i[i!=0].tolist() for i in newdata if i[i!=0].tolist()]
transactions = newdata
transactions

In [ ]:
from apyori import apriori
rules = apriori(transactions, min_support=0.00030,min_confidence = 0.05,min_lift = 2,min_length = 2)
results = list(rules)
results

In [ ]:
ordered_results_day = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])
ordered_results.sort_values(by =['Confidence', 'Lift'], ascending = False).head(20)